In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import odeint

# Define the system of ODEs
def system_ode(y, t, b, D1, D2, D3, Beta, Alpha, Mu, Eta):
    S, I1, I2, R = y
    dS_dt = b - (Beta * S * I1) - (D1 * S)
    dI1_dt = (Beta * S * I1) - ((D1 + D2 + Alpha) * I1) + (Eta * I1)
    dI2_dt = (Alpha * I1) - ((D2 + D3 + D3 + Mu) * I2)
    dR_dt = (Mu * I2) - ((D1 + D2) * R)
    return [dS_dt, dI1_dt, dI2_dt, dR_dt]

# Generate synthetic data
def generate_data(params, initial_conditions, t):
    return odeint(system_ode, initial_conditions, t, args=params)

# Normalize data
def normalize(data, data_min=None, data_max=None):
    if data_min is None or data_max is None:
        data_min, data_max = np.min(data, axis=0), np.max(data, axis=0)
    return (data - data_min) / (data_max - data_min), data_min, data_max

# Denormalize data
def denormalize(data, data_min, data_max):
    return data * (data_max - data_min) + data_min

# Define Swish ReLU activation function
def swish_relu(x):
    return x * tf.nn.sigmoid(x) + tf.nn.relu(x)  # Swish + ReLU
Parameters and initial conditions
params = (0.03, 0.01, 0.03, 0.03, 0.01, 0.095, 0.90, 0.03)
initial_conditions = [100, 5, 1, 0.0]
t = np.linspace(0, 50, 500)

# Generate data and normalize
synthetic_data = generate_data(params, initial_conditions, t)
synthetic_data, y_min, y_max = normalize(synthetic_data)
t, t_min, t_max = normalize(t.reshape(-1, 1))

# Convert to tensors
X_train = tf.convert_to_tensor(t, dtype=tf.float32)
Y_train = tf.convert_to_tensor(synthetic_data, dtype=tf.float32)

# Training function with early stopping
def train_pinn(model, X_train, Y_train, epochs=5000, learning_rate=1e-5, patience=500):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss_history = []
    best_loss, wait = float("inf"), 0  # Early stopping criteria

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            predictions = model(X_train)
            loss = tf.reduce_mean(tf.square(predictions - Y_train))  # MSE Loss

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        loss_history.append(loss.numpy())

        # Print progress every 100 epochs
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.numpy()}")

        # Early stopping condition
        if loss.numpy() < best_loss:
            best_loss = loss.numpy()
            wait = 0
        else:
            wait += 1
            if wait > patience:
                print(f"Early stopping at epoch {epoch}")
                break

    return loss_history

# Mean Squared Error (MSE)
def mean_squared_error(actual, predicted):
    return np.mean(np.square(actual - predicted))

# Run 20 trials and store MSE values for each compartment
num_trials = 20
mse_values_S, mse_values_I1, mse_values_I2, mse_values_R = [], [], [], []

for trial in range(num_trials):
    print(f"\n=== Trial {trial + 1}/{num_trials} ===")
    model = PINN()
    train_pinn(model, X_train, Y_train, epochs=20000, learning_rate=1e-5)  # Early stopping inside

    predictions = model(X_train).numpy()
    predictions_denorm = denormalize(predictions, y_min, y_max)
    synthetic_data_denorm = denormalize(Y_train.numpy(), y_min, y_max)

    # Compute MSE values
    mse_values_S.append(mean_squared_error(synthetic_data_denorm[:, 0], predictions_denorm[:, 0]))
    mse_values_I1.append(mean_squared_error(synthetic_data_denorm[:, 1], predictions_denorm[:, 1]))
    mse_values_I2.append(mean_squared_error(synthetic_data_denorm[:, 2], predictions_denorm[:, 2]))
    mse_values_R.append(mean_squared_error(synthetic_data_denorm[:, 3], predictions_denorm[:, 3]))

# Create a DataFrame for MSE values
mse_results = {
    "S (Susceptible)": mse_values_S,
    "I1 (Infected 1)": mse_values_I1,
    "I2 (Infected 2)": mse_values_I2,
    "R (Recovered)": mse_values_R
}

mse_df = pd.DataFrame(mse_results, index=[f"Trial {i+1}" for i in range(num_trials)])

# Print MSE values table
print("\n=== MSE Values for Each Compartment Across Trials ===")
print(mse_df.to_string())

# Plot MSE convergence for each compartment
plt.figure(figsize=(12, 8))
plt.plot(range(1, num_trials + 1), mse_values_S, marker='o', linestyle='--', label='S (Susceptible)', color='b')
plt.plot(range(1, num_trials + 1), mse_values_I1, marker='o', linestyle='--', label='I1 (Infected 1)', color='r')
plt.plot(range(1, num_trials + 1), mse_values_I2, marker='o', linestyle='--', label='I2 (Infected 2)', color='g')
plt.plot(range(1, num_trials + 1), mse_values_R, marker='o', linestyle='--', label='R (Recovered)', color='purple')

plt.xlabel("Trial Number")
plt.ylabel("Mean Squared Error (MSE)")
plt.title("MSE Convergence Across Trials for Each Compartment")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import odeint

# Define the system of ODEs
def system_ode(y, t, b, D1, D2, D3, Beta, Alpha, Mu, Eta):
    S, I1, I2, R = y
    dS_dt = b - (Beta * S * I1) - (D1 * S)
    dI1_dt = (Beta * S * I1) - ((D1 + D2 + Alpha) * I1) + (Eta * I1)
    dI2_dt = (Alpha * I1) - ((D2 + D3 + D3 + Mu) * I2)
    dR_dt = (Mu * I2) - ((D1 + D2) * R)
    return [dS_dt, dI1_dt, dI2_dt, dR_dt]

# Generate synthetic data
def generate_data(params, initial_conditions, t):
    return odeint(system_ode, initial_conditions, t, args=params)

# Normalize data
def normalize(data, data_min=None, data_max=None):
    if data_min is None or data_max is None:
        data_min, data_max = np.min(data, axis=0), np.max(data, axis=0)
    return (data - data_min) / (data_max - data_min), data_min, data_max

# Denormalize data
def denormalize(data, data_min, data_max):
    return data * (data_max - data_min) + data_min

# Define Swish ReLU activation function
def swish_relu(x):
    return x * tf.nn.sigmoid(x) + tf.nn.relu(x)  # Swish + ReLU

In [ ]:
# Parameters and initial conditions
params = (0.03, 0.01, 0.03, 0.03, 0.01, 0.095, 0.90, 0.03)
initial_conditions = [100, 5, 1, 0.0]
t = np.linspace(0, 50, 500)

# Generate data and normalize
synthetic_data = generate_data(params, initial_conditions, t)
synthetic_data, y_min, y_max = normalize(synthetic_data)
t, t_min, t_max = normalize(t.reshape(-1, 1))

# Convert to tensors
X_train = tf.convert_to_tensor(t, dtype=tf.float32)
Y_train = tf.convert_to_tensor(synthetic_data, dtype=tf.float32)

# Training function with early stopping
def train_pinn(model, X_train, Y_train, epochs=5000, learning_rate=1e-5, patience=500):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss_history = []
    best_loss, wait = float("inf"), 0  # Early stopping criteria

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            predictions = model(X_train)
            loss = tf.reduce_mean(tf.abs(predictions - Y_train))  # MAD Loss

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        loss_history.append(loss.numpy())

        # Print progress every 100 epochs
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.numpy()}")

        # Early stopping condition
        if loss.numpy() < best_loss:
            best_loss = loss.numpy()
            wait = 0
        else:
            wait += 1
            if wait > patience:
                print(f"Early stopping at epoch {epoch}")
                break

    return loss_history

# Mean Absolute Deviation (MAD)
def mean_absolute_deviation(actual, predicted):
    return np.mean(np.abs(actual - predicted))

# Run 20 trials and store MAD values for each compartment
num_trials = 20
mad_values_S, mad_values_I1, mad_values_I2, mad_values_R = [], [], [], []

for trial in range(num_trials):
    print(f"\n=== Trial {trial + 1}/{num_trials} ===")
    model = PINN()
    train_pinn(model, X_train, Y_train, epochs=20000, learning_rate=1e-5)  # Early stopping inside

    predictions = model(X_train).numpy()
    predictions_denorm = denormalize(predictions, y_min, y_max)
    synthetic_data_denorm = denormalize(Y_train.numpy(), y_min, y_max)

    # Compute MAD values
    mad_values_S.append(mean_absolute_deviation(synthetic_data_denorm[:, 0], predictions_denorm[:, 0]))
    mad_values_I1.append(mean_absolute_deviation(synthetic_data_denorm[:, 1], predictions_denorm[:, 1]))
    mad_values_I2.append(mean_absolute_deviation(synthetic_data_denorm[:, 2], predictions_denorm[:, 2]))
    mad_values_R.append(mean_absolute_deviation(synthetic_data_denorm[:, 3], predictions_denorm[:, 3]))

# Create a DataFrame for MAD values
mad_results = {
    "S (Susceptible)": mad_values_S,
    "I1 (Infected 1)": mad_values_I1,
    "I2 (Infected 2)": mad_values_I2,
    "R (Recovered)": mad_values_R
}

mad_df = pd.DataFrame(mad_results, index=[f"Trial {i+1}" for i in range(num_trials)])

# Print MAD values table
print("\n=== MAD Values for Each Compartment Across Trials ===")
print(mad_df.to_string())

# Plot MAD convergence for each compartment
plt.figure(figsize=(12, 8))
plt.plot(range(1, num_trials + 1), mad_values_S, marker='o', linestyle='--', label='S (Susceptible)', color='b')
plt.plot(range(1, num_trials + 1), mad_values_I1, marker='o', linestyle='--', label='I1 (Infected 1)', color='r')
plt.plot(range(1, num_trials + 1), mad_values_I2, marker='o', linestyle='--', label='I2 (Infected 2)', color='g')
plt.plot(range(1, num_trials + 1), mad_values_R, marker='o', linestyle='--', label='R (Recovered)', color='purple')

plt.xlabel("Trial Number")
plt.ylabel("Mean Absolute Deviation (MAD)")
plt.title("MAD Convergence Across Trials for Each Compartment")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import odeint

# Define the system of ODEs
def system_ode(y, t, b, D1, D2, D3, Beta, Alpha, Mu, Eta):
    S, I1, I2, R = y
    dS_dt = b - (Beta * S * I1) - (D1 * S)
    dI1_dt = (Beta * S * I1) - ((D1 + D2 + Alpha) * I1) + (Eta * I1)
    dI2_dt = (Alpha * I1) - ((D2 + D3 + D3 + Mu) * I2)
    dR_dt = (Mu * I2) - ((D1 + D2) * R)
    return [dS_dt, dI1_dt, dI2_dt, dR_dt]

# Generate synthetic data
def generate_data(params, initial_conditions, t):
    return odeint(system_ode, initial_conditions, t, args=params)

# Normalize data
def normalize(data, data_min=None, data_max=None):
    if data_min is None or data_max is None:
        data_min, data_max = np.min(data, axis=0), np.max(data, axis=0)
    return (data - data_min) / (data_max - data_min), data_min, data_max

# Denormalize data
def denormalize(data, data_min, data_max):
    return data * (data_max - data_min) + data_min

# Define Swish ReLU activation function
def swish_relu(x):
    return x * tf.nn.sigmoid(x) + tf.nn.relu(x)

# Parameters and initial conditions
params = (0.03, 0.01, 0.03, 0.03, 0.01, 0.095, 0.90, 0.03)
initial_conditions = [100, 5, 1, 0.0]
t = np.linspace(0, 50, 500)

# Generate data and normalize
synthetic_data = generate_data(params, initial_conditions, t)
synthetic_data, y_min, y_max = normalize(synthetic_data)
t, t_min, t_max = normalize(t.reshape(-1, 1))

# Convert to tensors
X_train = tf.convert_to_tensor(t, dtype=tf.float32)
Y_train = tf.convert_to_tensor(synthetic_data, dtype=tf.float32)

# Training function with early stopping
def train_pinn(model, X_train, Y_train, epochs=5000, learning_rate=1e-5, patience=500):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss_history = []
    best_loss, wait = float("inf"), 0

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            predictions = model(X_train)
            loss = tf.reduce_mean(tf.abs(predictions - Y_train))  # Using MAD loss

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        loss_history.append(loss.numpy())

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.numpy()}")

        # Early stopping
        if loss.numpy() < best_loss:
            best_loss = loss.numpy()
            wait = 0
        else:
            wait += 1
            if wait > patience:
                print(f"Early stopping at epoch {epoch}")
                break

    return loss_history

# Theil's Inequality Coefficient (TIC)
def theil_inequality_coefficient(actual, predicted):
    num = np.sqrt(np.mean((actual - predicted) ** 2))  # Root Mean Square Error
    denom = np.sqrt(np.mean(actual ** 2)) + np.sqrt(np.mean(predicted ** 2))
    return num / denom if denom != 0 else np.nan

# Run 20 trials and store TIC values
num_trials = 20
tic_values_S, tic_values_I1, tic_values_I2, tic_values_R = [], [], [], []

for trial in range(num_trials):
    print(f"\n=== Trial {trial + 1}/{num_trials} ===")
    model = PINN()
    train_pinn(model, X_train, Y_train, epochs=20000, learning_rate=1e-5)

    predictions = model(X_train).numpy()
    predictions_denorm = denormalize(predictions, y_min, y_max)
    synthetic_data_denorm = denormalize(Y_train.numpy(), y_min, y_max)

    # Compute Theil's Inequality Coefficient (TIC)
    tic_values_S.append(theil_inequality_coefficient(synthetic_data_denorm[:, 0], predictions_denorm[:, 0]))
    tic_values_I1.append(theil_inequality_coefficient(synthetic_data_denorm[:, 1], predictions_denorm[:, 1]))
    tic_values_I2.append(theil_inequality_coefficient(synthetic_data_denorm[:, 2], predictions_denorm[:, 2]))
    tic_values_R.append(theil_inequality_coefficient(synthetic_data_denorm[:, 3], predictions_denorm[:, 3]))

# Create a DataFrame for TIC values
tic_results = {
    "S (Susceptible)": tic_values_S,
    "I1 (Infected 1)": tic_values_I1,
    "I2 (Infected 2)": tic_values_I2,
    "R (Recovered)": tic_values_R
}

tic_df = pd.DataFrame(tic_results, index=[f"Trial {i+1}" for i in range(num_trials)])

# Print TIC values table
print("\n=== Theil's Inequality Coefficient for Each Compartment Across Trials ===")
print(tic_df.to_string())

# Plot TIC convergence for each compartment
plt.figure(figsize=(12, 8))
plt.plot(range(1, num_trials + 1), tic_values_S, marker='o', linestyle='--', label='S (Susceptible)', color='b')
plt.plot(range(1, num_trials + 1), tic_values_I1, marker='o', linestyle='--', label='I1 (Infected 1)', color='r')
plt.plot(range(1, num_trials + 1), tic_values_I2, marker='o', linestyle='--', label='I2 (Infected 2)', color='g')
plt.plot(range(1, num_trials + 1), tic_values_R, marker='o', linestyle='--', label='R (Recovered)', color='purple')

plt.xlabel("Trial Number")
plt.ylabel("Theil's Inequality Coefficient (TIC)")
plt.title("TIC Convergence Across Trials for Each Compartment")
plt.legend()
plt.grid(True)
plt.show()